# SHAP feature selection #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_bootstraps = 5 # this sets global setting of which how many bootstraps to use

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'max_leaves': 20,
    'max_depth': 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'min_data_in_leaf': 6,
    'lambda_l1': 0.9,
    'lambda_l2': 0.9,
    "verbose": -1
}

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from typing import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *
from yspecies.workflow import *
from yspecies.partition import *
from yspecies.selection import *

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import shap
from pprint import pprint
import random
import numpy as np
import lightgbm as lgb
from scipy.stats import kendalltau
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score, f1_score

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Loading data ###

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [8]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(452, 12630)",12630,43,452,"(12630, 2)","(44, 18)"


In [9]:
from sklearn.pipeline import Pipeline
selection = SelectedFeatures([],[], to_predict = "lifespan")
pipe = Pipeline([
    ('extractor', DataExtractor(selection)), 
    ("partitioner", DataPartitioner(species_in_validation=2)), 
    ("shap_computation", ShapSelector(ModelFactory()))]
)

In [10]:
d = pipe.fit_transform(data)
d

Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[241]	valid_0's l2: 14.2356	valid_0's l1: 2.17964
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[166]	valid_0's l2: 12.8786	valid_0's l1: 1.96514
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[210]	valid_0's l2: 19.9141	valid_0's l1: 2.36435
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[204]	valid_0's l2: 4.26647	valid_0's l1: 1.41393
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[21]	valid_0's l2: 373.983	valid_0's l1: 13.2287


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


MEAN metrics = R^2    0.782
MSE   85.056
MAE    4.230
dtype: float64


FeatureResults(weights=                   symbol  gain_score_to_lifespan    shap  \
ensembl_id                                                  
ENSG00000175854      SWI5              279113.218  67.546   
ENSG00000226979       LTA              197811.618   5.250   
ENSG00000060762      MPC1               88248.397 -70.060   
ENSG00000167515  TRAPPC2L               80191.721 -19.506   
ENSG00000010219     DYRK4               43076.047  15.146   
ENSG00000136436  CALCOCO2                7710.909  22.560   
ENSG00000120992    LYPLA1                6590.922  18.548   
ENSG00000129187      DCTD                5448.962   5.260   

                 kendall_tau_to_lifespan  
ensembl_id                                
ENSG00000175854                   -0.668  
ENSG00000226979                    0.028  
ENSG00000060762                   -0.199  
ENSG00000167515                   -0.305  
ENSG00000010219                    0.264  
ENSG00000136436                    0.797  
ENSG00000120992                   -0.442  
ENSG00000129187                    0.679  , stable_shap_values=array([[ 0.00148867,  0.        , -0.00481712, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00148867,  0.        , -0.00025443, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00148867,  0.        , -0.00025443, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.00084876,  0.        , -0.00025443, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00084876,  0.        , -0.00025443, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00084876,  0.        , -0.00025443, ...,  0.        ,
         0.        ,  0.        ]]), metrics=     R^2     MSE    MAE
0  0.981  14.236  2.180
1  0.984  12.879  1.965
2  0.965  19.914  2.364
3  0.994   4.266  1.414
4 -0.013 373.983 13.229)

In [12]:
#shap.summary_plot('mass_g', shap_values, df)
#shap.summary_plot(shap_values, df, feature_names=shap_feature_names, sort=False, plot_type='dot', max_display=100, show=False)
pipe.fit_transform(data)

Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[329]	valid_0's l2: 13.925	valid_0's l1: 2.08957
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[278]	valid_0's l2: 16.2151	valid_0's l1: 2.27646
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[284]	valid_0's l2: 15.0505	valid_0's l1: 2.32935
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[236]	valid_0's l2: 17.9281	valid_0's l1: 2.3675
Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[164]	valid_0's l2: 757.376	valid_0's l1: 12.9911


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


MEAN metrics = R^2     0.092
MSE   164.099
MAE     4.411
dtype: float64


FeatureResults(weights=                   symbol  gain_score_to_lifespan    shap  \
ensembl_id                                                  
ENSG00000175854      SWI5              295215.966  49.959   
ENSG00000060762      MPC1               98518.725 -91.705   
ENSG00000167515  TRAPPC2L               74403.568 -30.898   
ENSG00000066923     STAG3               46875.758  25.416   
ENSG00000164879       CA3               11046.768   6.120   
ENSG00000173567    ADGRF3                9564.103  37.771   
ENSG00000134489      HRH4                7922.326   9.717   
ENSG00000129187      DCTD                4958.421   0.655   
ENSG00000074621   SLC24A1                4157.085  -3.102   
ENSG00000118600    RXYLT1                3944.464  26.152   
ENSG00000172893     DHCR7                2122.127   2.849   
ENSG00000213139     CRYGS                1540.794   5.249   
ENSG00000165555   NOXRED1                1503.306  -0.575   
ENSG00000101353     MROH8                1275.164  -4.419   
ENSG00000204220     PFDN6                1157.960  -0.350   
ENSG00000165501      LRR1                1059.797   2.710   
ENSG00000152705  CATSPER3                 897.434   6.146   
ENSG00000162746     FCRLB                 648.637  -3.109   
ENSG00000116489    CAPZA1                 571.621  -4.246   
ENSG00000140025   EFCAB11                  62.888   0.356   

                 kendall_tau_to_lifespan  
ensembl_id                                
ENSG00000175854                   -0.686  
ENSG00000060762                   -0.189  
ENSG00000167515                   -0.223  
ENSG00000066923                    0.486  
ENSG00000164879                   -0.561  
ENSG00000173567                    0.491  
ENSG00000134489                    0.589  
ENSG00000129187                    0.682  
ENSG00000074621                    0.545  
ENSG00000118600                   -0.724  
ENSG00000172893                   -0.495  
ENSG00000213139                    0.494  
ENSG00000165555                    0.794  
ENSG00000101353                    0.658  
ENSG00000204220                   -0.533  
ENSG00000165501                    0.439  
ENSG00000152705                    0.529  
ENSG00000162746                    0.457  
ENSG00000116489                    0.217  
ENSG00000140025                    0.563  , stable_shap_values=array([[0.00018829, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00045162, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00045162, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00018829, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00045162, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00045162, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]]), metrics=     R^2     MSE    MAE
0  0.955  13.925  2.090
1  0.946  16.215  2.276
2  0.947  15.051  2.329
3  0.938  17.928  2.368
4 -3.325 757.376 12.991)